# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,6.25,93,75,4.12,CL,1714062499
1,1,port-aux-francais,-49.3500,70.2167,4.81,72,17,5.52,TF,1714062495
2,2,adamstown,-25.0660,-130.1015,23.18,90,7,8.82,PN,1714062486
3,3,flying fish cove,-10.4217,105.6791,26.82,89,75,4.63,CX,1714062413
4,4,blackmans bay,-43.0167,147.3167,10.96,75,95,0.45,AU,1714062484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    frame_width = 800,
    frame_height = 600,
    scale = 0.8,
    color = "City",
    size = "Humidity",
    tiles = "OSM")

# Display the map
city_map

C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 21) &
                                (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] < 10)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
96,96,santiago,-33.4569,-70.6483,21.27,34,0,2.57,CL,1714062326
119,119,quang ngai,15.1167,108.8000,26.22,75,6,1.60,VN,1714062608
190,190,greenville,35.6127,-77.3663,23.96,57,0,3.09,US,1714062634
204,204,menongue,-14.6585,17.6910,24.40,34,5,2.32,AO,1714062558
230,230,senanga,-16.1167,23.2667,23.97,20,0,1.73,ZM,1714062647
271,271,hun,29.1268,15.9477,24.00,25,4,4.28,LY,1714062664
290,290,maamba,-17.3667,27.1500,22.90,35,0,1.64,ZM,1714062671
313,313,tete,-16.1564,33.5867,25.87,47,0,1.74,MZ,1714062596
322,322,kalabo,-14.9700,22.6814,23.20,24,0,1.27,ZM,1714062682
364,364,fort bragg,35.1390,-79.0060,24.34,54,0,3.09,US,1714062488


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df.index.names = ['City_ID']
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

C:\Users\dsfac\AppData\Local\Temp\ipykernel_4604\2216790917.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
96,santiago,CL,-33.4569,-70.6483,34,
119,quang ngai,VN,15.1167,108.8000,75,
190,greenville,US,35.6127,-77.3663,57,
204,menongue,AO,-14.6585,17.6910,34,
230,senanga,ZM,-16.1167,23.2667,20,
271,hun,LY,29.1268,15.9477,25,
290,maamba,ZM,-17.3667,27.1500,35,
313,tete,MZ,-16.1564,33.5867,47,
322,kalabo,ZM,-14.9700,22.6814,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
filter = ""
bias = ""

params = {
    "categories":category,
    "filter":filter,
    "bias":bias,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santiago - nearest hotel: apart arturo prat
quang ngai - nearest hotel: Ngoc Anh
greenville - nearest hotel: Quality Inn
menongue - nearest hotel: No hotel found
senanga - nearest hotel: Senanga Safari Lodge
hun - nearest hotel: فندق الهروج
maamba - nearest hotel: No hotel found
tete - nearest hotel: Zambezi Hotel
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
fort bragg - nearest hotel: Airborne Inn Lodging
limulunga - nearest hotel: St. Lawrence Lodge
ilam - nearest hotel: Hotel Pokhereli
kalibo (poblacion) - nearest hotel: Hotel Simone
lang son - nearest hotel: No hotel found
jinghong - nearest hotel: โรงแรมสืเบา


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
96,santiago,CL,-33.4569,-70.6483,34,apart arturo prat
119,quang ngai,VN,15.1167,108.8000,75,Ngoc Anh
190,greenville,US,35.6127,-77.3663,57,Quality Inn
204,menongue,AO,-14.6585,17.6910,34,No hotel found
230,senanga,ZM,-16.1167,23.2667,20,Senanga Safari Lodge
271,hun,LY,29.1268,15.9477,25,فندق الهروج
290,maamba,ZM,-17.3667,27.1500,35,No hotel found
313,tete,MZ,-16.1564,33.5867,47,Zambezi Hotel
322,kalabo,ZM,-14.9700,22.6814,24,Golden Lodge Luxury Acomodation


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Hotel Name", "Country"],
    size = 70,
    color = "City",
    tiles = "OSM")

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)